# Scrape data for Japanese Translation Activity project

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests



In [17]:
GRAMMAR_POINTS_URL = "https://bunpro.jp/grammar_points"
headers = {}

grammar_points_page = requests.get(GRAMMAR_POINTS_URL)

soup = BeautifulSoup(grammar_points_page.text, 'html.parser')

grammar_points_list = []

for child in soup.find_all(attrs={"class":"u-position_absolute u-inset_0"}, recursive=True):
    grammar_points_list.append(child.attrs["href"])




In [19]:
print(len(grammar_points_list))

938
